In [2]:
import pandas as pd
import numpy as np
import datetime

In [3]:
budgetExcel = pd.concat(
    pd.read_excel("TALOUSARVIO 23.xlsx", sheet_name=None, header=[ 10, 11]),
    ignore_index=True,
)
if len(budgetExcel.columns) != 31:
    raise Exception("Sheets don't follow the same pattern")
# Dropping columns we don't need
budgetExcel = budgetExcel.drop(
    budgetExcel.columns[[3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]],
    axis=1,
)
# replace yes and no in the housing column
budgetExcel.iloc[:, 2] = budgetExcel.iloc[:, 2].map({"K": True, "E": False})

# CHANGE THIS HERE TO ONLY GET PROJECTS AND NOT MISS ANY PROJECT
budgetExcel = budgetExcel[budgetExcel[budgetExcel.columns[15]].notna() | budgetExcel[budgetExcel.columns[1]].notna()| budgetExcel[budgetExcel.columns[2]].notna() | budgetExcel[budgetExcel.columns[14]].notna()]
budgetExcel = budgetExcel.replace({np.nan: None, "?": None})


# currYear = datetime.date.today().year
# financeShift = list(budget_23.columns.get_level_values(0))[4] - currYear - 1
# if financeShift != 0:
#     finance_df = budget_23.iloc[:, 4:14]
#     budget_23.iloc[:, 4:14] = finance_df.shift(financeShift, axis=1)

# projects with Ids available
budgetExcel_Ids=budgetExcel[budgetExcel[budgetExcel.columns[15]].apply(type)==int]
# projects with no Ids available
budgetExcel_no_Ids = budgetExcel[budgetExcel[budgetExcel.columns[15]].apply(type)!=int]


In [4]:
planExcel = pd.concat(
    pd.read_excel("TOIMINTASUUNNITELMA 23.xlsx", sheet_name=None, header=[0, 1]),
    ignore_index=True,
)
if len(planExcel.columns) != 28:
    raise Exception("Sheets don't follow the same pattern")

planExcel = planExcel.drop(
    planExcel.columns[
        [5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
    ],
    axis=1,
)

# Replacing all common " with related network and project Ids
planExcel.iloc[:, 1] = planExcel.iloc[:, 1].dropna().replace(["\""], method="ffill").reindex(planExcel.iloc[:, 1].index)
planExcel.iloc[:, 2] = planExcel.iloc[:, 2].dropna().replace(["\""], method="ffill").reindex(planExcel.iloc[:, 2].index)

# Dropping all non project rows
planExcel = planExcel[planExcel[planExcel.columns[1]].notna() | planExcel[planExcel.columns[2]].notna() | planExcel[planExcel.columns[3]].notna()| planExcel[planExcel.columns[4]].notna() | planExcel[planExcel.columns[7]].notna()]
# # Strip spaces and capitalize
planExcel.iloc[:,3]=planExcel.iloc[:,3].str.strip().str.capitalize()
planExcel.iloc[:,4]=planExcel.iloc[:,4].str.strip().str.capitalize()
planExcel = planExcel.replace({np.nan: None, "?": None})

# plan_23.groupby(plan_23.columns[7]).aggregate({plan_23.columns[1]: lambda x: list(x)})




In [8]:
# merging budget projects with plan projects on same PW id
merged_Plan_Budget = pd.merge(
    budgetExcel_Ids,
    planExcel,
    left_on=[budgetExcel_Ids.columns[15]],
    right_on=[planExcel.columns[7]],
    how="inner",
)
# Fixing data discrepancy between similar columns
merged_Plan_Budget.iloc[:, 4] = merged_Plan_Budget.iloc[:, 21]
merged_Plan_Budget.iloc[:, 0] = merged_Plan_Budget.iloc[:, 16]
merged_Plan_Budget.iloc[:, 14] = merged_Plan_Budget.iloc[:, 22]
# Dropping duplicated columns
merged_Plan_Budget = merged_Plan_Budget.drop(
    merged_Plan_Budget.columns[[22, 21, 16]],
    axis=1,
)

notMerged_Budget = budgetExcel_Ids[
    ~budgetExcel_Ids[budgetExcel_Ids.columns[15]].isin(planExcel[planExcel.columns[7]])
]
notMerged_Budget = pd.concat([notMerged_Budget, budgetExcel_no_Ids])
notMerged_Plan = planExcel[
    ~planExcel[planExcel.columns[7]].isin(budgetExcel_Ids[budgetExcel_Ids.columns[15]])
]





In [9]:
notMerged_Budget_data = [
    project
    for project in zip(
        notMerged_Budget.iloc[:, 0],
        notMerged_Budget.iloc[:, 1],
        notMerged_Budget.iloc[:, 2],
        notMerged_Budget.iloc[:, 3],
        notMerged_Budget.iloc[:, 4],
        notMerged_Budget.iloc[:, 5],
        notMerged_Budget.iloc[:, 6],
        notMerged_Budget.iloc[:, 7],
        notMerged_Budget.iloc[:, 8],
        notMerged_Budget.iloc[:, 9],
        notMerged_Budget.iloc[:, 10],
        notMerged_Budget.iloc[:, 11],
        notMerged_Budget.iloc[:, 12],
        notMerged_Budget.iloc[:, 13],
        notMerged_Budget.iloc[:, 14],
        notMerged_Budget.iloc[:, 15],
    )
]



In [10]:
notMerged_Plan_data = [
    person
    for person in zip(
        notMerged_Plan.iloc[:, 0],
        notMerged_Plan.iloc[:, 1],
        notMerged_Plan.iloc[:, 2],
        notMerged_Plan.iloc[:, 3],
        notMerged_Plan.iloc[:, 4],
        notMerged_Plan.iloc[:, 5],
        notMerged_Plan.iloc[:, 6],
        notMerged_Plan.iloc[:, 7],
       
    )
]

In [11]:
merged_Plan_Budget_data = [
    project
    for project in zip(
        merged_Plan_Budget.iloc[:, 0],
        merged_Plan_Budget.iloc[:, 1],
        merged_Plan_Budget.iloc[:, 2],
        merged_Plan_Budget.iloc[:, 3],
        merged_Plan_Budget.iloc[:, 4],
        merged_Plan_Budget.iloc[:, 5],
        merged_Plan_Budget.iloc[:, 6],
        merged_Plan_Budget.iloc[:, 7],
        merged_Plan_Budget.iloc[:, 8],
        merged_Plan_Budget.iloc[:, 9],
        merged_Plan_Budget.iloc[:, 10],
        merged_Plan_Budget.iloc[:, 11],
        merged_Plan_Budget.iloc[:, 12],
        merged_Plan_Budget.iloc[:, 13],
        merged_Plan_Budget.iloc[:, 14],
        merged_Plan_Budget.iloc[:, 15],
        merged_Plan_Budget.iloc[:, 16],
        merged_Plan_Budget.iloc[:, 17],
        merged_Plan_Budget.iloc[:, 18],
        merged_Plan_Budget.iloc[:, 19],
    )
]



In [16]:
merged_Plan_Budget

Unnamed: 0_level_0 Unnamed: 1_level_0  \
                                   Unnamed: 0_level_1 Unnamed: 1_level_1   
0       Siltasaarenkatu 2-3 linja (Siltasaarenportti)                 K1   
1                                       Helsinginkuja                 K1   
2          Koskelan sairaalan ja pesuloiden alue (AM)               K5.1   
3                                 - Kunnalliskodintie               K5.1   
4      - Koskelantie (Käpyläntie - Kunnalliskodintie)               K5.1   
5                                        - Käpyläntie               K5.1   
6                                    - Koskelanakseli               K5.1   
7                                    - Paviljonkikuja               K5.1   
8                                  - Antti Korpin tie               K5.1   
9                                     - Kappelinkulku               K5.1   
10                                         Leanportti                 K1   
11                                         Paturintie               K5.1   
12  Kivipadontie (Maapadontie-Käskynhaltijantien r...               K5.1   
13  Pirjontien ja Pirkkolantien alue (AM/Raide-Jok...               None   
14     Maunulantie välillä Pirkkolantie-Maunulanpolku               K5.1   
15                                        Pirjonaukio               K5.1   
16                              - Lehtotielle hidaste               K5.1   
17  Metsäpurontie - Koivikkotie -Pirkkolantie + Kä...               K5.1   
18         Metsäpurontie (Pakilantie- Rajametsäntie)                K5.1   
19  Lampuotilantie itä (kääntöpaikka) + Keski jatk...               K5.1   
20          Käskynhaltijantien alue (AM/Raide-Jokeri)               None   
21                                       Maapadontie                K5.1   
22                                     Kivalterinkuja               K5.1   
23  Teinintie (sis. Kinkeripolku, ei länsiosaa lop...               K5.1   
24                                      Mestarintien                K5.1   
25               Maaherrantien alue (AM/Raide-Jokeri)               None   
26  Larin Kyöstin tie (-polku, Veräjänaukio ja -po...               K5.1   
27  Larin Kyöstin tie (-polku, Veräjänaukio ja -po...               K5.1   
28    Länsi-Pakilan liikennejärjestelyt (Bagge hanke)                 K4   
29                                    Viikinmäki (AM)               K5.1   
30                                        Harjannetie               K5.1   
31                              Ristiretkeläistenkatu               K5.1   
32  Säterintien alue: Säterintie, Säterinportti, M...               K5.1   
33         Maatullinpuiston eteläosan kaava-alue (AM)               None   
34                                 Rintamasotilaantie                 K4   
35                                           Rahkatie                 K4   
36                                         Takalantie                 K4   
37                                     Lähdeniityntie                 K4   
38                        Fallkullan kiilan alue (AM)               None   
39                                  Jokipoikasenkaari               K5.1   
40                                    Jokipoikasentie               K5.1   
41            Joutsentie (Tasankotie - Joutsenraitti)               K5.1   
42                                   Jäkäläpolku (LP)               K5.1   
43             Jäkälätie (v Jäkäläpolku - Joutsentie)               K5.1   
44                                         Sammaltori               K5.1   
45                                       Smoltinkaari               K5.1   
46                                        Smoltinkuja               K5.1   
47                                       Smoltinkulku               K5.1   
48                    Siltalanpuiston kaava-alue (AM)               K5.1   
49                                   Kiertotähdenkuja               K5.1   

   asuntotuotan. ennuste   2023   2024   2025   2026   2027   2028   2029  \
     